In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from datetime import timedelta
from src.constants import string_date_format, extreme_threshold, breakout_threshold
from src.market_profile import calculate_momentum_ranks, scan_momentum_ranks, plot_scan_results, calculate_tpo, calculate_value_area, rank_momentum
from src.utils import get_last_market_day, import_watchlist, get_market_hours
from src.pull_data import pull_watchlist_data

# Testing

In [2]:
watchlist = import_watchlist(os.path.abspath(r"files/watchlist.txt"))
last_market_day = get_last_market_day()
starting_day = last_market_day - timedelta(days=30)
df_raw = pull_watchlist_data(watchlist, start=starting_day, end=last_market_day+timedelta(days=1))

[*********************100%***********************]  89 of 89 completed


In [ ]:
momentum = calculate_momentum_ranks(df_raw)
scan = scan_momentum_ranks(momentum)

In [ ]:
df_momentum

In [ ]:
from src.utils import reaggregate_bars

In [ ]:
import plotly.graph_objects as go
df_symbol = df_raw[symbol.split(':')[0]]
fig = go.Figure(
    data=go.Candlestick(
        x=df_symbol.index,
        open=df_symbol['Open'],
        high=df_symbol['High'],
        low=df_symbol['Low'],
        close=df_symbol['Close']
    ),
)
fig.update_layout(title=dict(text=symbol))
fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_combined_charts(df_momentum:pd.DataFrame, df_symbol: pd.DataFrame, symbol: str):
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.6, 0.4],
        subplot_titles=(f"{symbol} OHLC", "Momentum Bar Chart")
    )

    # OHLC Chart
    fig.add_trace(
        go.Candlestick(
            x=df_symbol.index,
            open=df_symbol['Open'],
            high=df_symbol['High'],
            low=df_symbol['Low'],
            close=df_symbol['Close'],
            name='OHLC'
        ),
        row=1, col=1
    )

    # Momentum Bar Chart
    fig.add_trace(
        go.Bar(
            x=df_momentum.index,
            y=df_momentum[symbol],
            name='Momentum',
            text=df_momentum[symbol],
            textposition="auto"
        ),
        row=2, col=1
    )

    # Add horizontal lines to bar chart
    for y, color in zip([5, -5, 9, -9], ['blue', 'blue', 'red', 'red']):
        fig.add_shape(
            type="line",
            x0=df_momentum.index.min(), x1=df_momentum.index.max(),
            y0=y, y1=y,
            line=dict(color=color, width=0.5, dash="dot"),
            row=2, col=1
        )

    # Layout updates
    fig.update_yaxes(range=[-11, 11], row=2, col=1)
    fig.update_layout(
        height=700,
        showlegend=False
    )
    fig.update(layout_xaxis_rangeslider_visible=False)
    return fig


In [ ]:
available_dates = (
    pd.Series(df_raw.index.date).sort_values(ascending=True).unique().tolist()
)

In [ ]:
from datetime import datetime
from src.constants import string_date_format

In [ ]:
from src.constants import string_date_format
previous_date = available_dates[available_dates.index(datetime.strptime('2025-07-24', string_date_format).date()) - 1]

In [ ]:
for symbol in scan.columns:
    df_symbol = reaggregate_bars(df_raw[symbol.split(':')[0]])
    plot = plot_combined_charts(scan, df_symbol, symbol)
    plot.show()